In [20]:
import os
import csv
import json

from wrapper import read, pretty_file_name, solver, mapf, base_ipf, pe_simple_threshold, global_heatmap, partial_solving, select_portion_based_on_heatmap
from os import path
from multiprocessing import Pool

In [21]:
instances_pool_path = 'instances'
instances_pool = [(pretty_file_name(file), read(path.join(instances_pool_path, file))) for file in os.listdir(
    instances_pool_path) if path.isfile(path.join(instances_pool_path, file))]


timeout = 60 * 5
header = ['Step Name', 'Total Time', 'Ground Time', 'Solving Time', 'SAT', 'Additional Data']

# row = [{
#     'Step Name' : ,
#     'Total Time' :,
#     'Ground Time': ,
#     'Solving Time': ,
#     'SAT' : ,
#     'Additional Data' :
# }]


In [22]:
def write_csv(filename:str,header:list[str],rows):
    with open(file=filename, mode='w') as benchmark_csv:
        benchmark_writer = csv.DictWriter(benchmark_csv, delimiter=';', quotechar='"', quoting=csv.QUOTE_MINIMAL, fieldnames=header)
        benchmark_writer.writeheader()
        for row in rows:
            benchmark_writer.writerow(row)
            


In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        
        print(instance_name)
        r, s = mapf(instance_content)

        row = [{
            'Step Name' : 'base_mapf',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        }]
        
        print(bool(r))
        write_csv(f'benchmarks_result/{instance_name}-base_mapf.csv', header,row)

    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()
    

room_s29_a10_w5_h29
Grounding...


KeyboardInterrupt: 

Solving...
True


## Approach 1 : No additional path, heatmap simple threshold elimination, pre-computed path on full graph

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        # print(r)
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        

        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-approach01.csv', header,rows)

        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.59it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 35.22it/s]


...

SAT
749 13 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:01<00:00, 15.67it/s]


SAT
2002 32 0.05
[((1, 5), 78), ((21, 20), 65), ((15, 26), 64), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((17, 16), 48), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((1, 20), 37), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((1, 20), 34), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((2, 19), 22), ((8, 29), 21), ((21, 22), 13), ((13, 17), 29)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 26.86it/s]


SAT
167 1 0.2
[((13, 4), 7)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 2: No additional path, heatmap simple threshold elimination, subgraph approach

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        # r,s = solver(r,'mapf/add_lowest_heatmap_path.lp')
        
        print()
        # print(r)
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach02.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.33it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True


random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 32.40it/s]


SAT
569 24 0.06666666666666667
[((16, 6), 8), ((1, 2), 21), ((1, 2), 20), ((1, 2), 19), ((1, 2), 18), ((1, 2), 17), ((1, 2), 16), ((1, 2), 15), ((2, 2), 14), ((18, 3), 13), ((17, 3), 12), ((14, 9), 11), ((17, 7), 4), ((17, 7), 3), ((17, 7), 2), ((16, 3), 11), ((16, 4), 10), ((16, 7), 7), ((16, 5), 9), ((16, 8), 6), ((16, 7), 6), ((13, 7), 21), ((17, 7), 5), ((16, 8), 25)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True


maze_s10_a20_h40
40
Grounding...


...

SAT
749 13 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True


maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:01<00:00, 15.84it/s]


SAT
2002 32 0.05
[((1, 5), 78), ((21, 20), 65), ((15, 26), 64), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((17, 16), 48), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((1, 20), 37), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((1, 20), 34), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((2, 19), 22), ((8, 29), 21), ((21, 22), 13), ((13, 17), 29)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True


room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 29.40it/s]


SAT
167 1 0.2
[((13, 4), 7)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True




## Approach 3: no additional paths, simple threshold elimination, precomputed path on subgraph

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,           
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        print()
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        write_csv(f'benchmarks_result/{instance_name}-approach03.csv', header,rows)
        
    
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.09it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 36.45it/s]


...

SAT
749 13 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:01<00:00, 15.82it/s]


SAT
2002 32 0.05
[((1, 5), 78), ((21, 20), 65), ((15, 26), 64), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((17, 16), 48), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((1, 20), 37), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((1, 20), 34), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((2, 19), 22), ((8, 29), 21), ((21, 22), 13), ((13, 17), 29)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 30.11it/s]


SAT
167 1 0.2
[((13, 4), 7)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 4 : additional path, heatmap simple threshold elimination, pre-computed path on full graph

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        

        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
                
        print('\n\n\n')
        write_csv(f'benchmarks_result/{instance_name}-approach04.csv', header,rows)
        
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])
    # mutex.get(timeout=timeout)
    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print('Error',e)
        pool.terminate()

room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.14it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 33.52it/s]


SAT
585 24 0.06666666666666667
[((16, 6), 8), ((1, 2), 21), ((1, 2), 20), ((1, 2), 19), ((1, 2), 18), ((1, 2), 17), ((1, 2), 16), ((1, 2), 15), ((2, 2), 14), ((18, 3), 13), ((17, 3), 12), ((14, 9), 11), ((17, 7), 4), ((17, 7), 3), ((17, 7), 2), ((16, 3), 11), ((16, 4), 10), ((16, 7), 7), ((16, 5), 9), ((16, 8), 6), ((16, 7), 6), ((13, 7), 21), ((17, 7), 5), ((16, 8), 25)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is False
Grounding...
Solving...
Done
Solver is False
Grounding...
Solving...
Done
Solver is False
Grounding...
Solving...
Done
Solver is False




maze_s10_a20_h40
40
Grounding...


...

SAT
798 14 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 3), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:02<00:00,  7.70it/s]


SAT
2620 102 0.05
[((21, 20), 65), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((1, 5), 80), ((15, 26), 66), ((17, 16), 50), ((9, 17), 48), ((10, 17), 47), ((11, 17), 46), ((12, 17), 45), ((13, 17), 44), ((14, 17), 43), ((15, 17), 42), ((15, 18), 41), ((15, 19), 40), ((16, 19), 39), ((17, 19), 38), ((17, 18), 37), ((1, 20), 36), ((17, 17), 36), ((17, 16), 35), ((17, 15), 34), ((17, 14), 33), ((17, 13), 32), ((17, 12), 31), ((17, 11), 30), ((17, 10), 29), ((17, 9), 28), ((16, 9), 27), ((15, 9), 26), ((15, 10), 25), ((15, 11), 24), ((14, 11), 23), ((13, 11), 22), ((13, 12), 21), ((13, 13), 20), ((12, 13), 19), ((11, 13), 18), ((11, 12), 17), ((11, 11), 16), ((11, 10), 15), ((11, 9), 14), ((

Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 23.76it/s]


SAT
208 3 0.2
[((13, 5), 8), ((13, 4), 7), ((12, 4), 6)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True






## Approach 5:  additional path, heatmap simple threshold elimination, subgraph approach

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-approach05.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29
Grounding...

Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.61it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 32.54it/s]


SAT
585 24 0.06666666666666667
[((16, 6), 8), ((1, 2), 21), ((1, 2), 20), ((1, 2), 19), ((1, 2), 18), ((1, 2), 17), ((1, 2), 16), ((1, 2), 15), ((2, 2), 14), ((18, 3), 13), ((17, 3), 12), ((14, 9), 11), ((17, 7), 4), ((17, 7), 3), ((17, 7), 2), ((16, 3), 11), ((16, 4), 10), ((16, 7), 7), ((16, 5), 9), ((16, 8), 6), ((16, 7), 6), ((13, 7), 21), ((17, 7), 5), ((16, 8), 25)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
maze_s10_a20_h40
40
Grounding...


...

SAT
798 14 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 3), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:02<00:00,  7.77it/s]


SAT
2620 102 0.05
[((21, 20), 65), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((1, 5), 80), ((15, 26), 66), ((17, 16), 50), ((9, 17), 48), ((10, 17), 47), ((11, 17), 46), ((12, 17), 45), ((13, 17), 44), ((14, 17), 43), ((15, 17), 42), ((15, 18), 41), ((15, 19), 40), ((16, 19), 39), ((17, 19), 38), ((17, 18), 37), ((1, 20), 36), ((17, 17), 36), ((17, 16), 35), ((17, 15), 34), ((17, 14), 33), ((17, 13), 32), ((17, 12), 31), ((17, 11), 30), ((17, 10), 29), ((17, 9), 28), ((16, 9), 27), ((15, 9), 26), ((15, 10), 25), ((15, 11), 24), ((14, 11), 23), ((13, 11), 22), ((13, 12), 21), ((13, 13), 20), ((12, 13), 19), ((11, 13), 18), ((11, 12), 17), ((11, 11), 16), ((11, 10), 15), ((11, 9), 14), ((

Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 25.55it/s]


SAT
208 3 0.2
[((13, 5), 8), ((13, 4), 7), ((12, 4), 6)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 6: no additional paths, simple threshold elimination, precomputed path on subgraph

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        write_csv(f'benchmarks_result/{instance_name}-approach06.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.06it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 33.79it/s]


...

SAT
798 14 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 3), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:02<00:00,  7.68it/s]


SAT
2620 102 0.05
[((21, 20), 65), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((1, 5), 80), ((15, 26), 66), ((17, 16), 50), ((9, 17), 48), ((10, 17), 47), ((11, 17), 46), ((12, 17), 45), ((13, 17), 44), ((14, 17), 43), ((15, 17), 42), ((15, 18), 41), ((15, 19), 40), ((16, 19), 39), ((17, 19), 38), ((17, 18), 37), ((1, 20), 36), ((17, 17), 36), ((17, 16), 35), ((17, 15), 34), ((17, 14), 33), ((17, 13), 32), ((17, 12), 31), ((17, 11), 30), ((17, 10), 29), ((17, 9), 28), ((16, 9), 27), ((15, 9), 26), ((15, 10), 25), ((15, 11), 24), ((14, 11), 23), ((13, 11), 22), ((13, 12), 21), ((13, 13), 20), ((12, 13), 19), ((11, 13), 18), ((11, 12), 17), ((11, 11), 16), ((11, 10), 15), ((11, 9), 14), ((

Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 24.71it/s]


SAT
208 3 0.2
[((13, 5), 8), ((13, 4), 7), ((12, 4), 6)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 7: no additional paths, simple threshold elimination, subgraph with corridor

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/corridor.lp', additional_encodings=['strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach07.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.71it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 36.11it/s]


SAT
569 24 0.06666666666666667
[((16, 6), 8), ((1, 2), 21), ((1, 2), 20), ((1, 2), 19), ((1, 2), 18), ((1, 2), 17), ((1, 2), 16), ((1, 2), 15), ((2, 2), 14), ((18, 3), 13), ((17, 3), 12), ((14, 9), 11), ((17, 7), 4), ((17, 7), 3), ((17, 7), 2), ((16, 3), 11), ((16, 4), 10), ((16, 7), 7), ((16, 5), 9), ((16, 8), 6), ((16, 7), 6), ((13, 7), 21), ((17, 7), 5), ((16, 8), 25)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


maze_s10_a20_h40
40
Grounding...


...

SAT
749 13 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:01<00:00, 15.74it/s]


SAT
2002 32 0.05
[((1, 5), 78), ((21, 20), 65), ((15, 26), 64), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((17, 16), 48), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((1, 20), 37), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((1, 20), 34), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((2, 19), 22), ((8, 29), 21), ((21, 22), 13), ((13, 17), 29)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 31.29it/s]


SAT
167 1 0.2
[((13, 4), 7)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 8: no additional paths, simple threshold elimination, subgraph with diamond

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach08.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.33it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 32.64it/s]


...

SAT
749 13 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:01<00:00, 16.00it/s]


SAT
2002 32 0.05
[((1, 5), 78), ((21, 20), 65), ((15, 26), 64), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((17, 16), 48), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((1, 20), 37), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((1, 20), 34), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((2, 19), 22), ((8, 29), 21), ((21, 22), 13), ((13, 17), 29)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 31.35it/s]


SAT
167 1 0.2
[((13, 4), 7)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 9: no additional paths, simple threshold elimination, subgraph with corridor and diamond

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/corridor.lp', 'strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach09.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.22it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 34.66it/s]


...

SAT
749 13 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:01<00:00, 15.62it/s]


SAT
2002 32 0.05
[((1, 5), 78), ((21, 20), 65), ((15, 26), 64), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((17, 16), 48), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((1, 20), 37), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((1, 20), 34), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((2, 19), 22), ((8, 29), 21), ((21, 22), 13), ((13, 17), 29)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 31.53it/s]


SAT
167 1 0.2
[((13, 4), 7)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 10: additional paths, simple threshold elimination, precomputed path +  subgraph with corridor and diamond

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  pe_simple_threshold(ih,gh,s_ipf.additional_data['nagent'])
        
        rows.append({
            'Step Name' : 'simple_threshold',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/corridor.lp', 'strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })

        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach010.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 16.02it/s]


SAT
406 6 0.1
[((2, 25), 26), ((29, 19), 25), ((1, 29), 21), ((21, 17), 20), ((2, 29), 20), ((29, 18), 26)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


random_s20_a15_c75_h28
28
Grounding...


Current agent: 15 - at((17,7),0) => at((17,10),28): 100%|██████████| 15/15 [00:00<00:00, 33.86it/s]


...

SAT
798 14 0.05
[((24, 5), 11), ((24, 5), 10), ((24, 5), 9), ((24, 5), 8), ((24, 5), 7), ((24, 5), 6), ((22, 7), 24), ((22, 7), 23), ((24, 5), 12), ((29, 14), 16), ((29, 13), 15), ((24, 5), 13), ((25, 11), 3), ((25, 11), 2)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


maze_s15_a20_h98
98
Grounding...


Current agent: 20 - at((13,22),0) => at((15,22),98): 100%|██████████| 20/20 [00:02<00:00,  7.59it/s]


SAT
2620 102 0.05
[((21, 20), 65), ((20, 21), 63), ((21, 21), 50), ((20, 21), 49), ((19, 21), 48), ((18, 21), 47), ((17, 21), 46), ((16, 21), 45), ((15, 21), 44), ((14, 21), 43), ((13, 21), 42), ((12, 21), 41), ((11, 21), 40), ((10, 21), 39), ((9, 21), 38), ((9, 20), 37), ((9, 19), 36), ((9, 18), 35), ((9, 17), 34), ((10, 17), 33), ((11, 17), 32), ((12, 17), 31), ((13, 17), 30), ((1, 5), 80), ((15, 26), 66), ((17, 16), 50), ((9, 17), 48), ((10, 17), 47), ((11, 17), 46), ((12, 17), 45), ((13, 17), 44), ((14, 17), 43), ((15, 17), 42), ((15, 18), 41), ((15, 19), 40), ((16, 19), 39), ((17, 19), 38), ((17, 18), 37), ((1, 20), 36), ((17, 17), 36), ((17, 16), 35), ((17, 15), 34), ((17, 14), 33), ((17, 13), 32), ((17, 12), 31), ((17, 11), 30), ((17, 10), 29), ((17, 9), 28), ((16, 9), 27), ((15, 9), 26), ((15, 10), 25), ((15, 11), 24), ((14, 11), 23), ((13, 11), 22), ((13, 12), 21), ((13, 13), 20), ((12, 13), 19), ((11, 13), 18), ((11, 12), 17), ((11, 11), 16), ((11, 10), 15), ((11, 9), 14), ((

Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 25.18it/s]


SAT
208 3 0.2
[((13, 5), 8), ((13, 4), 7), ((12, 4), 6)]
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True




## Approach 11: no additional paths, summed heatmap elimination, precomputed path

In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        # print(r)
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        

        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph=False, precomputed_path='mapf/precomputed_path_allgraph.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-approach11.csv', header,rows)

        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e, e.with_traceback)
        pool.terminate()

room_s29_a10_w5_h29
29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 14.58it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Done
Solver is True
Grounding...
Solving...
Done
Solver is True
random_s20_a15_c75_h28
28
Grounding...


...

Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 25.79it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 12: No additional path,summed heatmap elimination, subgraph approach

In [ ]:


for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        # r,s = solver(r,'mapf/add_lowest_heatmap_path.lp')
        
        print()
        # print(r)
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach12.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29


29
Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.93it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Grounding...
Solving...
Done
Solver is True


room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 31.77it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Solving...
Done
Solver is True




## Approach 13: no additional paths, summed heatmap elimination, precomputed path on subgraph

In [ ]:

for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,           
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        print()
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp', horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        write_csv(f'benchmarks_result/{instance_name}-approach13.csv', header,rows)
        
    
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.71it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 25.69it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True

Grounding...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


## Approach 14:  additional path, summed heatmap elimination, subgraph approach

In [ ]:

for instance_name, instance_content in instances_pool:
    def approach(instance_name,instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])
        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False)
        
        rows.append({
            'Step Name' : 'partial_solving',

            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path=False,  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        

        write_csv(f'benchmarks_result/{instance_name}-approach14.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 16.21it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 25.39it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Solving...
Done
Solver is True


## Approach 15: no additional paths, summed heatmap elimination, precomputed path +  subgraph with corridor and diamond


In [ ]:
for instance_name, instance_content in instances_pool:
    def approach(instance_name, instance_content):
        print(instance_name)
        rows = []
        
        r, s_ipf = base_ipf(instance_content, additional_paths_computation=True)
        
        rows.append({
            'Step Name' : 'base_ipf',
            'Total Time' : s_ipf.total_time,
            'Ground Time': s_ipf.total_time - s_ipf.solve_time,
            'Solving Time': s_ipf.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s_ipf.additional_data)
        })
        
        ih, gh, lgh, s = global_heatmap(r + instance_content)
        
        rows.append({
            'Step Name' : 'heatmaps',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        cva ,s =  select_portion_based_on_heatmap(lgh['summed_heatmap_per_path'])

        
        rows.append({
            'Step Name' : 'select_portion_based_on_heatmap',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        r,s = solver(r + cva + instance_content + lgh['pwlsh'], 'pathelimination/path_elimination.lp', additional_encodings=['pathelimination/convert_to_kill.lp'])
        
        rows.append({
            'Step Name' : 'path_elimination',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        
        r,s = solver(r, 'pathselection/path_selection.lp', parameters=['0','--opt-strategy=usc'])
        
        rows.append({
            'Step Name' : 'path_selection',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })
        # print(r)
        
        r,s = solver(r,'strategies/diamond.lp', additional_encodings=['strategies/corridor.lp', 'strategies/determination.lp'])
        
        rows.append({
            'Step Name' : 'strategy',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r),
            'Additional Data' : json.dumps(s.additional_data)
        })

        r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp')
        
        rows.append({
            'Step Name' : 'partial_solving',
            'Total Time' : s.total_time,
            'Ground Time': s.total_time - s.solve_time,
            'Solving Time': s.solve_time,
            'SAT' : bool(r_),
            'Additional Data' : json.dumps(s.additional_data)
        })
        
        for i in range (1,6,2):
            if s.additional_data['ngoal_reached'] < s_ipf.additional_data['nagent']:
                r_,s = partial_solving(r, 'mapf/partial_solving.lp', subgraph='mapf/subgraph.lp', precomputed_path='mapf/precomputed_path.lp',  horizon_modifier=i)
        
                rows.append({
                    'Step Name' : f'partial_solving_additional_horizon',
                    'Total Time' : s.total_time,
                    'Ground Time': s.total_time - s.solve_time,
                    'Solving Time': s.solve_time,
                    'SAT' : bool(r_),
                    'Additional Data' : json.dumps(s.additional_data)
                })
        
        print('\n')
        write_csv(f'benchmarks_result/{instance_name}-approach015.csv', header,rows)
        
    pool = Pool(1)       
    mutex = pool.apply_async(approach,[instance_name,instance_content])

    try:
        mutex.get(timeout=timeout)
    except Exception as e:
        print(e)
        pool.terminate()

room_s29_a10_w5_h29
29


Grounding...


Current agent: 10 - at((12,10),0) => at((21,24),29): 100%|██████████| 10/10 [00:00<00:00, 15.70it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



...

Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


room_s19_a5_w4_h25
25
Grounding...


Current agent: 5 - at((15,2),0) => at((10,6),25): 100%|██████████| 5/5 [00:00<00:00, 24.72it/s]


SAT
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


pathelimination/path_elimination.lp:2:5-25: info: atom does not occur in any rule head:
  critical_vertex(V,T)

pathelimination/path_elimination.lp:3:5-26: info: atom does not occur in any rule head:
  critical_agent(R,V,T)



Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True
Grounding...
Solving...
Done
Solver is True


